In [17]:
from graph_tool.all import Graph, shortest_distance, graph_draw
from deap import base, creator, tools, algorithms
import python_codes.files_operators as fo
import numpy as np
import random

# 读取图和位置信息
filename1 = "Germany50"
read_graph, read_pos = fo.read_files(f"../networks_clusters/{filename1}.net")
print("nodes: ", read_graph.num_vertices(), "links ", read_graph.num_edges())

num_vertices = read_graph.num_vertices()
num_edges = read_graph.num_edges()

# 将图转换为个体编码
def graph_to_individual(graph):
    individual = [0] * (num_vertices * num_vertices)
    for edge in graph.edges():
        source, target = int(edge.source()), int(edge.target())
        individual[source * num_vertices + target] = 1
        individual[target * num_vertices + source] = 1
    return individual

# 将个体解码为图
def individual_to_graph(individual):
    graph = Graph(directed=False)
    graph.add_vertex(num_vertices)
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            if individual[i * num_vertices + j] == 1:
                graph.add_edge(graph.vertex(i), graph.vertex(j))
    return graph

# 初始化个体
def init_individual():
    individual = [0] * (num_vertices * num_vertices)
    edges = set()
    while len(edges) < num_edges:
        i, j = random.sample(range(num_vertices), 2)
        if i != j:
            edges.add((min(i, j), max(i, j)))  # 确保没有自环且每条边只被添加一次
    for i, j in edges:
        individual[i * num_vertices + j] = 1
        individual[j * num_vertices + i] = 1
    return creator.Individual(individual)

# 定义适应度函数
def evaluate(individual):
    graph = individual_to_graph(individual)
    dist_matrix = shortest_distance(graph).get_2d_array(range(graph.num_vertices()))
    total_distance = np.sum(dist_matrix[dist_matrix != np.inf])
    return total_distance,

# 保证变异后的个体边数不变
def mutate(individual):
    edges = [(i, j) for i in range(num_vertices) for j in range(i + 1, num_vertices) if individual[i * num_vertices + j] == 1]
    non_edges = [(i, j) for i in range(num_vertices) for j in range(i + 1, num_vertices) if individual[i * num_vertices + j] == 0]

    if edges and non_edges:
        edge_to_remove = random.choice(edges)
        non_edge_to_add = random.choice(non_edges)

        i, j = edge_to_remove
        k, l = non_edge_to_add

        # 确保要添加的边不重复
        while individual[k * num_vertices + l] == 1 or individual[l * num_vertices + k] == 1:
            non_edge_to_add = random.choice(non_edges)
            k, l = non_edge_to_add

        individual[i * num_vertices + j] = 0
        individual[j * num_vertices + i] = 0
        individual[k * num_vertices + l] = 1
        individual[l * num_vertices + k] = 1

    # 变异后检查边数
    if sum(individual) // 2 != num_edges:
        print(f"Error: Edge count mismatch after mutation. Expected: {num_edges}, Got: {sum(individual) // 2}")
    assert sum(individual) // 2 == num_edges, "Error: Edge count mismatch after mutation."

    return individual,

# 遗传算法设置
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

# 主遗传算法流程
def main():
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof, verbose=True)
    return pop, stats, hof

if __name__ == "__main__":
    pop, stats, hof = main()
    best_individual = hof[0]
    best_graph = individual_to_graph(best_individual)
    print("Final best graph: nodes: ", best_graph.num_vertices(), "links ", best_graph.num_edges())

    # 为每个节点标上对应的序号
    vertex_text = best_graph.new_vertex_property("string")
    for v in best_graph.vertices():
        vertex_text[v] = str(int(v))

    # 绘制最终优化后的图
    graph_draw(best_graph, read_pos, vertex_text=vertex_text, edge_color='blue', output_size=(1000, 1000))


nodes:  50 links  69
gen	nevals	avg        	std        	min  	max        
0  	300   	6.89543e+11	3.34611e+11	9,540	1.73087e+12
Error: Edge count mismatch after mutation. Expected: 69, Got: 67


AssertionError: Error: Edge count mismatch after mutation.